LLM application through query routing and long-context, a type of reflection flow based on the user query
===

RAG routing is based on the complexity of the query and the context. The routing follows these rules:

1. Simple query, direct answer, no routing.
2. Answerable question, but complex query, then routing for query decomposition.
3. Route to long-context query for unanswerable questions.

Origin paper: [Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach](https://arxiv.org/abs/2407.16833)

Impl. by: LLama-Index


In [1]:
from rich.pretty import pprint as pp
from icecream import ic
from IPython.display import Markdown

import nest_asyncio
nest_asyncio.apply()

## Const

In [2]:
from llama_index.core.response_synthesizers import ResponseMode

llm = "llama3-70b-8192" #"models/gemini-1.5-pro-exp-0801" # Gemini for "models/gemini-1.5-pro" # Grop for "llama3-70b-8192", "llama-3.1-70b-versatile", "llama-3.1-8b-instant" # openai for "gpt-3.5-turbo", "gpt-4o-mini"
embeds = "text-embedding-3-small" # Gemini for "models/text-embedding-004", openai for "text-embedding-3-large",  "text-embedding-3-small"
chunk_size = 512
MAX_TOKENS = 2048
data_urls = [
    "https://hermesworld.com/de/karriere/jobs/Junior-Manager-mwd-HR-Controlling-de-j4364.html",
    "https://hermesworld.com/de/karriere/jobs/Senior-Product-Manager-mwd-de-j3839.html",
]
rerank_top_k = 5
similarity_top_k = 5
num_multi_steps = 4
verbose = True
streaming = False
response_mode = ResponseMode.TREE_SUMMARIZE
# https://github.com/run-llama/llama_index/blob/71c2cfdfbec6bcdd71f0e39f5dbb52c0e9f68ae5/llama-index-core/llama_index/core/response_synthesizers/type.py

## Token counting

In [3]:
from llama_index.core.callbacks import TokenCountingHandler
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model( "gpt-4o-mini").encode,
    verbose=verbose,
)

In [4]:
import pandas as pd

def get_df_token_counter(counter: TokenCountingHandler) -> pd.DataFrame:
    data = {
        'Metric': ['Embedding Tokens', 'LLM Prompt Tokens', 'LLM Completion Tokens', 'Total LLM Token Count'],
        'Count': [counter.total_embedding_token_count, counter.prompt_llm_token_count, counter.completion_llm_token_count, counter.total_llm_token_count]
    }
    df = pd.DataFrame(data) 
    df.set_index('Metric', inplace=True)
    
    counter.reset_counts()
    return df

df = get_df_token_counter(token_counter)
df.head()

,Count
Metric,
Embedding Tokens,0
LLM Prompt Tokens,0
LLM Completion Tokens,0
Total LLM Token Count,0


## LLama-Index setting

In [5]:
from llama_index.core import Settings
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core.callbacks import CallbackManager
from llama_index.llms.groq import Groq
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = Groq(
    model=llm, temperature=0, max_tokens=MAX_TOKENS
)  
Settings.embed_model = OpenAIEmbedding(model_name=embeds) # GeminiEmbedding, OpenAIEmbedding
Settings.callback_manager  = CallbackManager([token_counter])

In [6]:
from llama_index.readers.web import SimpleWebPageReader
import tempfile
from llama_parse import LlamaParse

origin_documents = SimpleWebPageReader(html_to_text=True).load_data(data_urls)
# pp(origin_documents)
document_contents = "\n".join(
    [f"----document 1:----\n\n{doc.text}" for doc in origin_documents]
)
# print(document_contents)
temp_file = tempfile.NamedTemporaryFile(delete=True, suffix=".txt")
temp_file.write(document_contents.encode("utf-8"))
temp_file.seek(0) 
ic(temp_file.name)
print(temp_file.read()) 
parser = LlamaParse(
    # api_key= api_key,
    result_type="markdown"
) 
documents = parser.load_data(temp_file.name)
pp(documents)
temp_file.close()

ic| temp_file.name: '/tmp/tmpw2j74v6g.txt'


b'----document 1:----\n\n[ Privatkunden ](https://www.myhermes.de)\n\n\xc3\x96sterreich China Deutschland Vereinigtes K\xc3\xb6nigreich International USA\n\nMenu AT CN DE EN INT US [Hermes](https://www.hermesworld.com/de/)\n\n  * navigation.search\n\n  * [Start](https://www.hermesworld.com/de/)\n  * [Unsere Dienstleistungen](https://www.hermesworld.com/de/unsere-dienstleistungen/)\n\n    * [Transport Logistics](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/)\n      * [Supply Chain Solutions](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Unsere Fokusbranchen](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Supply Chain Services](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/supply-chain-services/)\n        * [E-Services](https://www.herme

[
│   Document(
│   │   id_='c0e35ecb-d130-430d-898c-fa6911db1237',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Privatkunden\n\nÖsterreich China Deutschland Vereinigtes Königreich International USA\n\nHermes\n\n# Menu\n\n- navigation.search\n- Start\n- Unsere Dienstleistungen\n- Transport Logistics\n- Unsere Fokusbranchen\n- Supply Chain Services\n- E-Services\n- Zoll Services\n- Nachhaltigkeit\n- Unser Partnernetzwerk\n- FML\n- Jeena\n- SEKO\n- WWG\n- Global E-Commerce\n- Global E-Commerce\n- Distribution\n- Paketversand\n- Paketversand\n- 2-Mann-Handling\n- 2-Mann-Handling\n- Individuelle Versandlösungen',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='3a4cdf79-305c-4daf-99e9-0c69b40d807f',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Zusammenarbeit\n\n- Zusammenarbeit\n- Dropshipping\n- Unser Service für Versender\n- Unser Service für Endkunden\n- CO2-neutraler Versand\n- HES Pay\n\n# Distribution in Europa\n\n- Distribution in Europa\n\n# Über uns\n\n- Über uns\n\n# Auf einen Blick\n\n- Hermes auf einen Blick\n\n# Hermes Gruppe\n\n- Hermes Gruppe\n\n# Hermes Einrichtungs Service\n\n- Hermes Einrichtungs Service\n- Logistikzentren\n- Auslieferdepots\n\n# Hermes Fulfilment\n\n- Hermes Fulfilment\n- Daten & Fakten\n- Einblicke\n- Nachhaltigkeit\n\n# Hermes Logistik Österreich\n\n- Hermes Logistik Österreich\n- PaketShop Partner\n\n# Hermes Germany GmbH\n\n- Hermes Germany GmbH\n- PaketShop-Partner werden\n- Vertragspartner werden\n- Formular\n\n# Hermes NexTec\n\n- Hermes NexTec\n\n# Hermes Germany\n\n- Hermes Germany',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='12055a23-c9a2-4f76-a5fa-7c6f2b7b2f7a',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# gruppe/hermes-transport-logistics/hermes-transport-logistics/\n\n- Hermes Germany\n- Downloads\n\n# Hermes UK\n\n- Hermes UK\n- Kundenstimmen\n\n# Hermes BorderGuru\n\n- Hermes BorderGuru\n\n# Management\n\n- Management\n\n# Klima- und Umweltschutz\n\n- Klima- und Umweltschutz\n- Grüne Zustellung\n- Energieeffiziente Standorte\n- Unterstützung von Klimaschutzprojekten\n\n# Soziale Verantwortung\n\n- Faire Zusammenarbeit\n- Einhaltung von Sozialstandards\n- Gesellschaftliches Engagement\n\n# Downloads\n\n- Plattformen & Bezahlfunktionen China\n- Export nach China\n\n# Messen\n\n- Messen\n\n# Historie\n\n- 2010er Jahre\n- 2000er Jahre\n- 1990er Jahre\n- 1980er Jahre\n- 1970er Jahre\n\n# Karriere\n\n- Karriere Übersicht\n- Jobs bei Hermes Germany\n- Jobbörse der Hermes-Gruppe\n- Unsere Gesellschaften',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='3bae75ac-d7dc-42d3-9582-48961f889ffe',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Hermes Fulfilment\n\n- Hermes Fulfilment\n- Was uns auszeichnet\n- Ausbildung\n- Studierende & Absolventen\n- Deine Bewerbung\n\n# Hermes Germany\n\n- Hermes Germany\n- Ausbildung & Duales Studium\n- Studierende & Absolventen\n- Berufserfahrene\n- Tech\n- Architecture & Information

## Setup step methods

### Chain

In [7]:
from llama_index.core import get_response_synthesizer

synthesizer = get_response_synthesizer(
    response_mode=response_mode, streaming=streaming
)

### Prompting

Re-prompt the default prompt for German input data. This can enhance query performance.

In [8]:
from langchain import hub
from llama_index.core import PromptTemplate

prompt_str = hub.pull("hwchase17/llama-rag").template.replace("context", "context_str").replace("question", "query_str")
prompt_str = prompt_str.replace("[INST] <<SYS>>", "[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. ")
pp(prompt_str)

prompt_tmpl = PromptTemplate(prompt_str)

'[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'

### Top K RAG

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import SentenceTransformerRerank


splitter = SentenceSplitter(chunk_size=chunk_size)
vector_index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=similarity_top_k,
    response_synthesizer=synthesizer,
    node_postprocessors=[
        SentenceTransformerRerank(top_n=rerank_top_k, model="BAAI/bge-reranker-base")
    ],
    summary_template=prompt_tmpl,
)
vector_query_engine.update_prompts(
    {"response_synthesizer:summary_template": prompt_tmpl}
)

Embedding Token Usage: 3504


In [10]:
prompts_dict = vector_query_engine.get_prompts()
pp(list(prompts_dict.values()))

[
│   PromptTemplate(
│   │   metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>},
│   │   template_vars=['context_str', 'query_str'],
│   │   kwargs={},
│   │   output_parser=None,
│   │   template_var_mappings=None,
│   │   function_mappings=None,
│   │   template='[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'
│   )
]

### Multi-step query

In [11]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.query_engine import MultiStepQueryEngine

step_decompose_transform = StepDecomposeQueryTransform(verbose=verbose)
multi_steps_query_engine = MultiStepQueryEngine(
    query_engine=vector_query_engine,
    query_transform=step_decompose_transform,
    response_synthesizer=synthesizer,
    num_steps=num_multi_steps,
)

### Long context query

In [12]:
from llama_index.core.llms.llm import LLM
from llama_index.core.query_engine import CustomQueryEngine
from typing import Any
from langchain import hub
from llama_index.core import PromptTemplate


class VanillaQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    llm: LLM
    context: str
    prompt_tmpl: PromptTemplate

    def __call__(self, *args: Any, **kwds: Any) -> str:
        return self.custom_query(*args, **kwds)

    def custom_query(self, query_str: str) -> str:
        full_query = self.prompt_tmpl.format(
            context_str=self.context, query_str=query_str
        )
        return str(self.llm.complete(full_query))


lc_query_engine = VanillaQueryEngine(
    response_synthesizer=synthesizer,
    llm=Settings.llm,
    context="\n".join([doc.text for doc in documents]),
    prompt_tmpl=prompt_tmpl,
)

## Query with methods

### Simple query for try

In [13]:
query = "Tell me the jobs you know at Hermes Group?"

### Top K RAG

In [14]:
vc_res = vector_query_engine.query(query)
print(vc_res.response)

Embedding Token Usage: 10


LLM Prompt Token Usage: 1554
LLM Completion Token Usage: 139
Bei Hermes Gruppe gibt es verschiedene Arbeitsplätze mit langfristigen Perspektiven. Einige der bekanntesten Jobs sind:

* (Junior) Manager (m/w/d) HR Controlling
* Jobs bei Hermes Germany
* Jobs beim Hermes Einrichtungs Service (HES)
* Jobs in unseren Depots
* Karrieremessen

Es gibt auch verschiedene Abteilungen wie Logistik & gewerbliche Jobs, Tech, Architecture & Information Security, ANS, Cloud & Application Infrastructure, IT-Finance und IT-Licensemanagement, BlueOps, European Network, Black OPs, Business Analytics, Support Solutions, Servicedesk, Supply Chain Solutions usw.


In [15]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,3514
LLM Prompt Tokens,1554
LLM Completion Tokens,139
Total LLM Token Count,1693


### Multi-step query

In [16]:
ms_res = multi_steps_query_engine.query(query)
print(ms_res.response)

LLM Prompt Token Usage: 444
LLM Completion Token Usage: 21


> Current query: Tell me the jobs you know at Hermes Group?
> New query: Since there is no knowledge source context provided, the next question would be:

What is Hermes Group?
Embedding Token Usage: 20
LLM Prompt Token Usage: 1073
LLM Completion Token Usage: 32
LLM Prompt Token Usage: 496
LLM Completion Token Usage: 12
> Current query: Tell me the jobs you know at Hermes Group?
> New query: The next question would be: What is Hermes Group?
Embedding Token Usage: 11
LLM Prompt Token Usage: 1292
LLM Completion Token Usage: 46
LLM Prompt Token Usage: 554
LLM Completion Token Usage: 32
> Current query: Tell me the jobs you know at Hermes Group?
> New query: Based on the context and previous reasoning, the next question would be:

What kind of jobs or Arbeitsplätze (workplaces) does Hermes Group offer?
Embedding Token Usage: 30
LLM Prompt Token Usage: 1854
LLM Completion Token Usage: 46
LLM Prompt Token Usage: 330
LLM Completion Token Usage: 54
LLM Prompt Token Usage: 210
LLM Completion To

In [17]:
def show_multi_steps(ms_res):
    sub_qa = ms_res.metadata["sub_qa"]
    tuples = [(t[0], t[1].response) for t in sub_qa]
    pp(tuples)

show_multi_steps(ms_res)

[
│   (
│   │   'Since there is no knowledge source context provided, the next question would be:\n\nWhat is Hermes Group?',
│   │   'Die Hermes Gruppe wird unter "# Über uns" erwähnt, aber leider gibt es keine weitere Information darüber, was die Hermes Gruppe genau ist.'
│   ),
│   (
│   │   'The next question would be: What is Hermes Group?',
│   │   'Die Hermes Gruppe bietet vielseitige Arbeitsplätze mit langfristigen Perspektiven und täglich wechselnden Herausforderungen in einer sicheren und zukunftsorientierten Branche.'
│   ),
│   (
│   │   'Based on the context and previous reasoning, the next question would be:\n\nWhat kind of jobs or Arbeitsplätze (workplaces) does Hermes Group offer?',
│   │   'Die Hermes Gruppe bietet eine Vielzahl von Arbeitsplätzen an, darunter Jobs bei Hermes Germany, Logistik- und gewerbliche Jobs, Jobs beim Hermes Einrichtungs Service (HES), Jobs in unseren Depots und viele mehr.'
│   ),
│   (
│   │   'What kind of jobs or Arbeitsplätze (workplaces) does Hermes Group offer in Hermes Germany, Logistik- und gewerbliche Jobs, Hermes Einrichtungs Service (HES), and Depots?',
│   │   'Die Hermes Gruppe bietet vielseitige Arbeitsplätze mit langfristigen Perspektiven und täglich wechselnden Herausforderungen in einer sicheren und zukunftsorientierten Branche. Im Kontext von Hermes Germany, Logistik- und gewerbliche Jobs, Hermes Einrichtungs Service (HES) und Depots werden folgende Arbeitsplätze angeboten:\n\n* Logistik- und gewerbliche Jobs\n* Jobs bei Hermes Germany\n* Jobs beim HES\n* Jobs beim HES in Ansbach\n* Jobs in unseren Depots\n* (Junior) Manager (m/w/d) HR Controlling\n\nEs gibt auch weitere Karrieremöglichkeiten in verschiedenen Bereichen, wie zum Beispiel in der IT, im Management, in der Logistik und vielem mehr.'
│   )
]

### Long context query

In [18]:
lc_res = lc_query_engine(query)
ic(lc_res)

LLM Prompt Token Usage: 3353
LLM Completion Token Usage: 75


ic| lc_res: ('Based on the provided context, I can identify the following job openings at '
             'Hermes Group:
            '
             '
            '
             '1. (Junior) Manager (m/w/d) HR Controlling
            '
             '2. (Senior-) Product Manager (m/w/d)
            '
             '
            '
             'Please note that these are the only job openings mentioned in the provided '
             'context. There might be other job openings available at Hermes Group that '
             'are not mentioned here.')


'Based on the provided context, I can identify the following job openings at Hermes Group:\n\n1. (Junior) Manager (m/w/d) HR Controlling\n2. (Senior-) Product Manager (m/w/d)\n\nPlease note that these are the only job openings mentioned in the provided context. There might be other job openings available at Hermes Group that are not mentioned here.'

# Replication query router

The routing is ochstrated by the complexity of the query and the context. Because of the complexity of the query, the RAG will be failed for some reason. The routing will be used to decompose the query into multiple steps. The long-context query will be used for unanswerable questions.

## RAG can be failed by 4 reasons

Accoriding to the paper, there are four reasons why RAG can be failed.

The four reasons include: 

(A) The query requires multi-step reasoning so the results of previous steps are needed to retrieve information for later steps, e.g. ‘‘What nationality is the performer of song XXX’’. 

(B) The query is general, e.g. ‘‘What does the group think about XXX’’, which is challenging for the retriever to formulate a good query. 

(C) The query is long and complex, which is challenging for the retriever to understand. However, answering this kind of questions is arguably, an advantage of LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context. For instance, in a lengthy conversational narrative about a space voyage, a question like ‘‘What caused the shadow behind the spaceship?’’ requires readers to connect the dots and deduce the answer, as there is no explicit mention of the shadow when the cause is revealed.

##### Short to say:

(A) The query requires multi-step reasoning.

(B) The query is general. 

(C) The query is long and complex, requiring LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context.



## Setup route query engine

### Tools

In [19]:
from llama_index.core.tools import QueryEngineTool

vector_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for simple and answerable questions.",
    return_direct=True,
)

multi_steps_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=multi_steps_query_engine,
    description="Useful for answerable questions that require multiple step reasonings.",
    return_direct=True,
)

lc_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=lc_query_engine,
    description="Useful for long and complex or implicit questions or queries that require full context to obtain results.",
    return_direct=True,
)

query_engine_tools = [
    vector_query_engine_tool,
    multi_steps_query_engine_tool,
    lc_query_engine_tool,
]

### Router query engine

#### Router selection prompting

The routing choice is determined by the query's complexity and context, which can trigger errors for various reasons. 

This serves as the prompt for conducting error analysis on the routing choice, we can modify the origin content and migrate to the Llama-Index [LLMSingleSelector](https://github.com/run-llama/llama_index/blob/71c2cfdfbec6bcdd71f0e39f5dbb52c0e9f68ae5/llama-index-core/llama_index/core/selectors/llm_selectors.py).

Origin prompt for the error analysis in paper:

```text

You are given some text chunks from an article, and a question. The text chunks are retrieved by an external retriever.
Now:
(1) Tell whether the question can be answered based only on the provided text chunks.
(2) If the question can be answered, answer the question based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question cannot be answered, choose the reason from the following:
A. The question needs multistep reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question is a general query, thus it is hard to retrieve relevant chunks. For example, "What did the group
think about Dave leaving?" is general because the group may include multiple persons, and they can have different
thinkings.
C. The question is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks. For
example, "What did Julie Morgan elaborate on the online survey when talking about the evaluations on the legitimacy
of the children’s rights, protection and demands?", "The Huskies football team were invited to the Alamo Bowl where
they were defeated by a team coached by Art Briles and who played their home games at what stadium?"
D. The question is not explicit and requires comprehensive understanding of the whole story and cannot be solved
using retrieval-augmented generation. For example, "What caused the shadow behind Koerber’s ship?" needs a
comprehensive understanding of the whole story. Another example like "How many words are there in the article"
also requires the complete article.
E. Others.
Keep the above reasons in mind, and choose the most possible reason if you think the question cannot be answered
based on the text. Output the results in JSON format.
{in_context_examples}
Text: {context}
Question: {input}
Answer:

```

We can modifiy this into:

In [20]:
modified_error_anlysis = """You receive text chunks from various data sources along with a question or query. These chunks are obtained through an external retriever or query engine.
Now:
(1) Think about whether the question or the query can be simply answered based only on the provided text chunks.
(2) If the question or the query can be answered, answer the question or the query based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question or the query cannot be answered, choose the reason from the following:
A. The question or the query needs multi-step reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question or the query is a general query, thus it is hard to retrieve relevant chunks. For example, "What did the group
think about Dave leaving?" is general because the group may include multiple persons, and they can have different
thinkings.
C. The question or the query is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks. For
example, "What did Julie Morgan elaborate on the online survey when talking about the evaluations on the legitimacy
of the children’s rights, protection and demands?", "The Huskies football team were invited to the Alamo Bowl where
they were defeated by a team coached by Art Briles and who played their home games at what stadium?"
D. The question or the query is not explicit and requires comprehensive understanding of the whole story and cannot be solved
using retrieval-augmented generation. For example, "What caused the shadow behind Koerber’s ship?" needs a
comprehensive understanding of the whole story. Another example like "How many words are there in the article"
also requires the complete article.
E. Others.

Keep the above reasons in mind, and choose the most possible reason(s) with following rule to handle the question or the query that cannot be answered
based on the text:

For error A and B and E, choose the multi-step query engine.
For error C and D, choose the query engine that can handle long and complex or implicit questions or queries.
Otherwise, just consider that the question or the query is simply answerable based on the provided text chunks.

Notice that text chunks are in German and the question or the query might be in English, please translate the it into German and think in German style if necessary.
Here is the instruction with choices for the selection task:
"""

In [21]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector
from llama_index.core.response_synthesizers import TreeSummarize

SINGLE_SELECT_PROMPT_TMPL = modified_error_anlysis + (
    "Some choices are given below. It is provided in a numbered list "
    "(1 to {num_choices}), "
    "where each item in the list corresponds to a summary.\n"
    "---------------------\n"
    "{context_list}"
    "\n---------------------\n"
    "Using only the choices above and not prior knowledge, return "
    "the choice that is most relevant to the question: '{query_str}'\n"
)

print(SINGLE_SELECT_PROMPT_TMPL)

router_query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(
        prompt_template_str=SINGLE_SELECT_PROMPT_TMPL
    ),
    query_engine_tools=query_engine_tools,
    summarizer=TreeSummarize(
        streaming=streaming,
        use_async=True,
        verbose=verbose,
    ),
    verbose=verbose,
)

You receive text chunks from various data sources along with a question or query. These chunks are obtained through an external retriever or query engine.
Now:
(1) Think about whether the question or the query can be simply answered based only on the provided text chunks.
(2) If the question or the query can be answered, answer the question or the query based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question or the query cannot be answered, choose the reason from the following:
A. The question or the query needs multi-step reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question or the query is a general query, thus it is

#### Simple question

In [22]:
simply_query = "What data do you have?"
route_res = router_query_engine.query(simply_query)
Markdown(route_res.response)

LLM Prompt Token Usage: 729
LLM Completion Token Usage: 43
Selecting query engine 0: The question 'What data do you have?' is a simple and direct question that can be answered based on the provided text chunks..
Embedding Token Usage: 6
LLM Prompt Token Usage: 1851
LLM Completion Token Usage: 172
LLM Prompt Token Usage: 286
LLM Completion Token Usage: 85
LLM Prompt Token Usage: 342
LLM Completion Token Usage: 109


Hallo! Ich habe Daten über eine Stelle als (Junior) Manager (m/w/d) HR Controlling bei der Hermes Germany GmbH und verschiedenen Dienstleistungen und Angeboten von Hermes Germany, einschließlich Tools und Services wie Sendungsverfolgung und PaketShop-Findung, sowie Informationen über die Architektur und Sicherheit, Logistik und gewerbliche Jobs, Hermes Europe, Hermes Einrichtungs Service, Karrieremessen, Presse und Mediathek, Kontakt und Karriere.

In [23]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,108
LLM Prompt Tokens,15588
LLM Completion Tokens,1105
Total LLM Token Count,16693


In [24]:
lc_res = lc_query_engine.query(simply_query)
Markdown(lc_res.response)

LLM Prompt Token Usage: 3349
LLM Completion Token Usage: 62


Hallo! Ich habe Daten über die Hermes Gruppe, einem Logistikunternehmen. Meine Daten umfassen Informationen über die Dienstleistungen, die Fokusbranchen, die Unternehmen der Hermes Gruppe, Karrierechancen, Newsroom, Kontaktinformationen und vieles mehr. Wenn du eine bestimmte Frage hast, stehe ich gerne zur Verfügung!

In [25]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,0
LLM Prompt Tokens,3349
LLM Completion Tokens,62
Total LLM Token Count,3411


#### Multi-step reasoning

In [26]:
normal_query = "Introduce (Senior) Product Manager or (Junior) Manager HR Controlling, return final result in a markdown table (two rows for each job and two columns for job title, task) respectively."  
route_res = router_query_engine.query(normal_query)
Markdown(route_res.response)

LLM Prompt Token Usage: 763
LLM Completion Token Usage: 36
Selecting query engine 1: The question requires multi-step reasoning to introduce the two job roles and their respective tasks..
LLM Prompt Token Usage: 504
LLM Completion Token Usage: 75
> Current query: Introduce (Senior) Product Manager or (Junior) Manager HR Controlling, return final result in a markdown table (two rows for each job and two columns for job title, task) respectively.
> New query: Since there is no knowledge source context provided, I would ask:

What are the typical tasks and responsibilities of a (Senior) Product Manager and a (Junior) Manager HR Controlling?

This question aims to gather more information about the job roles, which can then be used to create the markdown table with two rows for each job and two columns for job title and task.
Embedding Token Usage: 74
LLM Prompt Token Usage: 1854
LLM Completion Token Usage: 362
LLM Prompt Token Usage: 587
LLM Completion Token Usage: 86
LLM Prompt Token Usag

Here is the markdown table with two rows for each job and two columns for job title and task:

| Job Title | Tasks |
| --- | --- |
| (Senior-) Produktmanager | - Verantwortung für strategische und operative Produktportfolios über die gesamte Customer Journey<br>- Beobachtung und Analyse von Zielgruppen, Markt und Wettbewerb, um Produkt-, Preis- und Kommunikationsmaßnahmen zu initiieren und umzusetzen |
| (Junior-) Manager HR Controlling | - Erstellung von Auswertungen, Statistiken und Präsentationen für interne und externe Stakeholder<br>- Begleitung von Prozessen und HR-Projekten, um datenbasierte Entscheidungen treffen zu können |

In [27]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,131
LLM Prompt Tokens,10345
LLM Completion Tokens,1817
Total LLM Token Count,12162


In [32]:
lc_res = lc_query_engine.query(normal_query)
Markdown(lc_res.response)

LLM Prompt Token Usage: 3383
LLM Completion Token Usage: 188


Here are the job introductions in a markdown table:

| Job Title | Task |
| --- | --- |
| (Senior-) Product Manager | Du verantwortest strategisch und operativ einen Teil unseres Produktportfolios (z.B. Retoure) über die gesamte Customer Journey, von der Beauftragung durch den Versender bis zur Zustellung an den Empfänger |
|  | Du leitest neue Produktideen unter Berücksichtigung der Unternehmensziele ab und erstellst dafür Marktanalysen, Business Cases und Präsentationen mit klaren Handlungsempfehlungen |
| (Junior) Manager HR Controlling | Du erstellst Auswertungen, Statistiken und Präsentationen für interne Stakeholder wie HR Kolleg*innen oder Fachbereiche sowie für externe Stakeholder wie Behörden |
|  | Du begleitest tatkräftig Prozesse (z.B. Gehaltsrunde) sowie HR-Projekte, um datenbasierte Entscheidungen treffen zu können |

In [29]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,0
LLM Prompt Tokens,3383
LLM Completion Tokens,2048
Total LLM Token Count,5431


#### Long and complex question

In [30]:
long_complex_query="Tell me the information of the jobs within your knowledge base, including TODO and the benefits for working in the Hermes Group, response in a markdown table."
route_res = router_query_engine.query(long_complex_query)
Markdown(route_res.response)

LLM Prompt Token Usage: 753
LLM Completion Token Usage: 116
Selecting query engine 2: The question or the query is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks..
LLM Prompt Token Usage: 3373
LLM Completion Token Usage: 400


Here is the information about the jobs within the knowledge base, including the TODO and the benefits for working in the Hermes Group, in a markdown table:

| **Job Title** | **Company** | **Location** | **Job Description** | **Requirements** | **Benefits** |
| --- | --- | --- | --- | --- | --- |
| (Junior) Manager (m/w/d) HR Controlling | Hermes Germany GmbH | Hamburg | Create reports, statistics, and presentations for internal stakeholders; support processes and HR projects with data-based decisions | Studied in economics, social sciences, or personnel management; first work experience; proficiency in MS Office | Flexible working hours, mobile working possible; comprehensive training offer; 15% discount on otto.de and other shops; company pension scheme, job bike, various sports and leisure offers |
| (Senior-) Product Manager (m/w/d) | Hermes Germany GmbH | Hamburg | Responsible for strategic and operational product management; analyze target groups, market, and competition; initiate product launches | Studied in product management or similar; several years of experience in product or brand management; analytical skills, customer orientation, creativity, and negotiation skills | Flexible working hours, mobile working possible; comprehensive training offer; 15% discount on otto.de and other shops; company pension scheme, job bike, various sports and leisure offers |

**TODO:**

* There are only two job postings in the knowledge base. If more job postings are added, they can be included in the table.
* The job descriptions and requirements are summarized. If more details are needed, they can be added to the table.

**Benefits of working in the Hermes Group:**

* Flexible working hours, mobile working possible
* Comprehensive training offer
* 15% discount on otto.de and other shops
* Company pension scheme
* Job bike
* Various sports and leisure offers
* International company with a multicultural work atmosphere
* Opportunities for career growth and development

In [31]:
get_df_token_counter(token_counter)

,Count
Metric,
Embedding Tokens,0
LLM Prompt Tokens,4126
LLM Completion Tokens,516
Total LLM Token Count,4642
